This is to solve non trivial sudoku, i.e. the ones that cannot be soleved by simply follow the column/row/grid rules, but where we need to implement some more clever logic.

The idea is to get the probability of a number going in a specific position, guess it and then use a brute force approach

In [208]:
# imports
import numpy as np
import sys
import math
import time
from solverFunctions import *

In [209]:
# test grids
emptySudoku = np.zeros((9,9))

hardSudoku = np.zeros((9,9))
hardSudoku[0,1] = 4
hardSudoku[0,7] = 1
hardSudoku[1,0] = 2
hardSudoku[1,8] = 6
hardSudoku[7,0] = 9
hardSudoku[7,8] = 2
hardSudoku[8,1] = 1
hardSudoku[8,7] = 9

sampleSudoku = np.array([[5, 3, 0, 0, 7, 0, 0, 0, 0],
                         [6, 0, 0, 1, 9, 5, 0, 0, 0],
                         [0, 9, 8, 0, 0, 0, 0, 6, 0],
                         [8, 0, 0, 0, 6, 0, 0, 0, 3],
                         [4, 0, 0, 8, 0, 3, 0, 0, 1],
                         [7, 0, 0, 0, 2, 0, 0, 0, 6],
                         [0, 6, 0, 0, 0, 0, 2, 8, 0],

                         [0, 0, 0, 4, 1, 9, 0, 0, 5],
                         [0, 0, 0, 0, 8, 0, 0, 7, 9]])

In [210]:
numIs = 1

squaresIdx = ([[0, 1, 2], [0, 1, 2]],
              [[0, 1, 2], [3, 4, 5]],
              [[0, 1, 2], [6, 7, 8]],
              [[3, 4, 5], [0, 1, 2]],
              [[3, 4, 5], [3, 4, 5]],
              [[3, 4, 5], [6, 7, 8]],
              [[6, 7, 8], [0, 1, 2]],
              [[6, 7, 8], [3, 4, 5]],
              [[6, 7, 8], [6, 7, 8]]
             )

allProbs = []
allFrees = []
allPoss = []
for numProb in range(1,10):
    probSquare = np.zeros((3,3))
    intIdxR = 0
    for rIdx in squaresIdx[6][0]:
        intIdxC = 0;
        for cIdx in squaresIdx[6][1]:
            squareIdRow = getSquareID(rIdx)
            squareIdCol = getSquareID(cIdx)
            square = hardSudoku[squareIdRow: squareIdRow+3, squareIdCol:squareIdCol+3]
            line = np.ndarray.flatten(square)
            freeSpots =  len([i for i in line if i == 0])
            if probabilityFunction(hardSudoku, rIdx, cIdx, numProb):
                probSquare[intIdxR, intIdxC] = 1
            intIdxC = intIdxC + 1
        intIdxR = intIdxR + 1
        
    line = np.ndarray.flatten(probSquare)
    possibilities =  len([i for i in line if i > 0])
    allPoss.append(possibilities)
    allProbs.append(probSquare)
    allFrees.append(freeSpots)

For this square, the number 2 is the onr that can has less possible available spots. Now let's try to get all possible filled up squares that filled up this space respecting the existing constraints.

In [211]:
from itertools import permutations 

# Get all permutations 1-9
perm = list(permutations(list(range(1,10))))

keepPerm = []
for i in perm:
    if i[3] == 9 and i[7] == 1 and not i[0] == 2 and not i[4] == 2 and not i[5] == 2 and not i[6] == 2 and not i[1] == 4 and not i[4] == 4:
        keepPerm.append(i)

print(len(keepPerm))

print('done')

1560
done


Generalize function to give number of possible square arrangements

In [214]:
# wrapper to call all the the check funcitons
# copy of the existing one but supppressing prints
def validateGrid(grid):
    if not grid.shape == (9,9):
        return(False)
    rows = all(np.apply_along_axis(lineCheck, 0, grid))
    cols = all(np.apply_along_axis(lineCheck, 1, grid))
    if( not rows or not cols):
        return(False)
    gridCoord = [0,3,6]
    checkSquareRes = []
    for i in gridCoord:
        for ii in gridCoord:
            square = grid[i:i+3, ii:ii+3]
            checkSquareRes = np.append(checkSquareRes, checkSquare(square))
            
    
    checkSquareRes = all(checkSquareRes)
    if( not checkSquareRes):
        return(False)
    
    return(True)

def getNArrang(grid, selSquare):
    squaresIdx = ([[0, 1, 2], [0, 1, 2]],
                  [[0, 1, 2], [3, 4, 5]],
                  [[0, 1, 2], [6, 7, 8]],
                  [[3, 4, 5], [0, 1, 2]],
                  [[3, 4, 5], [3, 4, 5]],
                  [[3, 4, 5], [6, 7, 8]],
                  [[6, 7, 8], [0, 1, 2]],
                  [[6, 7, 8], [3, 4, 5]],
                  [[6, 7, 8], [6, 7, 8]]
             )

    # Get all permutations 1-9
    perm = list(permutations(list(range(1,10))))
    
    squareIdRow = getSquareID(squaresIdx[selSquare][0][0])
    squareIdCol = getSquareID(squaresIdx[selSquare][1][0])
    square = grid[squareIdRow: squareIdRow+3, squareIdCol:squareIdCol+3]
    line = np.ndarray.flatten(square)
    freeSpots =  len([i for i in line if i == 0])      
                
    #if(freeSpots == 9):
    #    return(perm)
    if (freeSpots == 0):
        return([])
    
    
    allProbs = []
    allFrees = []
    allPoss = []
    for numProb in range(1,10):
        probSquare = np.zeros((3,3))
        intIdxR = 0
        for rIdx in squaresIdx[selSquare][0]:
            intIdxC = 0;
            for cIdx in squaresIdx[selSquare][1]:              
                if probabilityFunction(grid, rIdx, cIdx, numProb):
                    probSquare[intIdxR, intIdxC] = 1
                intIdxC = intIdxC + 1
            intIdxR = intIdxR + 1      
        line = np.ndarray.flatten(probSquare)
        possibilities =  len([i for i in line if i > 0])
        allPoss.append(possibilities)
        allProbs.append(probSquare)
        allFrees.append(freeSpots)
    
    #get the first minimum values
    minVal = min(i for i in allPoss if i > 0)
    minPos = allPoss.index(minVal)
    triedNumb = minPos + 1
    probGrid = allProbs[minPos]
    probGrid[probGrid == 1] = triedNumb

    keepPerm = []
    line = np.ndarray.flatten(square)
    for i in perm:
        #use only permutations where existing numbers are where they are in the original square 
        diff = np.asarray(line)-np.asarray(i)
        inters = np.where(diff == 0)
        #how many numbers we have in origna grid
        fixedPos = 9 - freeSpots
        if len(inters[0]) == fixedPos: # only intersect the numbers we already have. All of them
            #keep only permutaitons where our number is in the desired positions
            
            diff = np.asarray(np.ndarray.flatten(probGrid))-np.asarray(i)
            inters = np.where(diff == 0)
            if len(inters[0]) == 1:  
                #we still need to eliminate permutations were a number ends up in
                #a row or columkn forbidden by the remaining
                #of the grid. Easiset way is to put the permutation in the main grid and check with
                #validateGrid()
                tempGrid = grid.copy()
                tempGrid[squareIdRow: squareIdRow+3, squareIdCol:squareIdCol+3] = np.asarray(i).reshape(3,3)
                if validateGrid(tempGrid):
                    keepPerm.append(i)

    return(keepPerm)

In [215]:
keepPerm = getNArrang(hardSudoku, 6)
print(len(keepPerm))

1560


In [216]:
#get the number of poss for each square
allPerms = []
for i in range(0,9):
    keepPerm = getNArrang(hardSudoku, i)
    allPerms.append(len(keepPerm))
    
print(allPerms)

KeyboardInterrupt: 

If it still cannot be solved, we keep with the brute force approach: replace the identified grid with on possibility, and restart from the beginning.

In [193]:
b = hardSudoku.copy()

keepPerm = getNArrang(hardSudoku, 6)
b[6:9, 0:3] = np.asarray(keepPerm[0]).reshape((3,3))
keepPerm = getNArrang(b, 0)
b[0:3, 0:3] = np.asarray(keepPerm[0]).reshape((3,3))
keepPerm = getNArrang(b, 2)
b[0:3, 6:9] = np.asarray(keepPerm[0]).reshape((3,3))
keepPerm = getNArrang(b, 8)
b[6:9, 6:9] = np.asarray(keepPerm[0]).reshape((3,3))
keepPerm = getNArrang(b, 1)
b[0:3, 3:6] = np.asarray(keepPerm[0]).reshape((3,3))
keepPerm = getNArrang(b, 3)
b[3:6, 0:3] = np.asarray(keepPerm[0]).reshape((3,3))
keepPerm = getNArrang(b, 5)
b[3:6, 6:9] = np.asarray(keepPerm[0]).reshape((3,3))
keepPerm = getNArrang(b, 7)
b[6:9, 3:6] = np.asarray(keepPerm[0]).reshape((3,3))
keepPerm = getNArrang(b, 4)
print(len(keepPerm))

0


We need to make the above a loop with nested for's until it is solved, but first let's see if we can vectorize or speed up things anyway.

In [237]:
def getNArrangFast(grid, selSquare):
    squaresIdx = ([[0, 1, 2], [0, 1, 2]],
                  [[0, 1, 2], [3, 4, 5]],
                  [[0, 1, 2], [6, 7, 8]],
                  [[3, 4, 5], [0, 1, 2]],
                  [[3, 4, 5], [3, 4, 5]],
                  [[3, 4, 5], [6, 7, 8]],
                  [[6, 7, 8], [0, 1, 2]],
                  [[6, 7, 8], [3, 4, 5]],
                  [[6, 7, 8], [6, 7, 8]]
             )

    # Get all permutations 1-9
    perm = list(permutations(list(range(1,10))))
    
    squareIdRow = getSquareID(squaresIdx[selSquare][0][0])
    squareIdCol = getSquareID(squaresIdx[selSquare][1][0])
    square = grid[squareIdRow: squareIdRow+3, squareIdCol:squareIdCol+3]
    line = np.ndarray.flatten(square)
    freeSpots =  len([i for i in line if i == 0])      
                
    if (freeSpots == 0):
        return([])
    
    
    allProbs = []
    allFrees = []
    allPoss = []
    for numProb in range(1,10):
        probSquare = np.zeros((3,3))
        intIdxR = 0
        for rIdx in squaresIdx[selSquare][0]:
            intIdxC = 0;
            for cIdx in squaresIdx[selSquare][1]:              
                if probabilityFunction(grid, rIdx, cIdx, numProb):
                    probSquare[intIdxR, intIdxC] = 1
                intIdxC = intIdxC + 1
            intIdxR = intIdxR + 1      
        line = np.ndarray.flatten(probSquare)
        possibilities =  len([i for i in line if i > 0])
        allPoss.append(possibilities)
        allProbs.append(probSquare)
        allFrees.append(freeSpots)
    
    #get the first minimum values
    minVal = min(i for i in allPoss if i > 0)
    minPos = allPoss.index(minVal)
    triedNumb = minPos + 1
    probGrid = allProbs[minPos]
    probGrid[probGrid == 1] = triedNumb

    keepPerm = []
    line = np.ndarray.flatten(square)
    for i in perm:
        #use only permutations where existing numbers are where they are in the original square 
        diff = np.asarray(line)-np.asarray(i)
        inters = np.where(diff == 0)
        #how many numbers we have in origna grid
        fixedPos = 9 - freeSpots
        if len(inters[0]) == fixedPos: # only intersect the numbers we already have. All of them
            #keep only permutaitons where our number is in the desired positions
            
            diff = np.asarray(np.ndarray.flatten(probGrid))-np.asarray(i)
            inters = np.where(diff == 0)
            if len(inters[0]) == 1:  
                #we still need to eliminate permutations were a number ends up in
                #a row or columkn forbidden by the remaining
                #of the grid. Easiset way is to put the permutation in the main grid and check with
                #validateGrid()
                # this is slow. might be faster to remove it entirely
                tempGrid = grid.copy()
                tempGrid[squareIdRow: squareIdRow+3, squareIdCol:squareIdCol+3] = np.asarray(i).reshape(3,3)
                if validateGrid(tempGrid):
                    keepPerm.append(i)

    return('keepPerm')

In [238]:
getNArrangFast(hardSudoku, 1)

'keepPerm'

In [234]:
perm = list(permutations(list(range(1,10))))
lenPerm = int(len(perm)/9)
perm = np.asarray(perm)
print(perm.shape)
line = np.ndarray.flatten(hardSudoku[0:3,6:9])
nVal = len([i for i in line if i > 0])
keepPerm = perm-line



SyntaxError: invalid syntax (<ipython-input-234-7c1471f213fd>, line 8)

In [226]:
362880/9

40320.0